In [1]:
import librosa as lr
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import logging

from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot  


class VoiceSamples(Dataset):
    
    def __init__(self, core_name, samples_path=None, Automatic=None):
        
        self.Log = logging.getLogger()
        logging.basicConfig(level=logging.INFO)
        
        self.noiseThreshold = 1
        
        self.core_name = core_name
        self.samples_path = samples_path
        
        self.soundSamples = []
        self.sampleRate = []
        self.path = []
        
        self.chopedSamples = []
        self.chopedSr = []
        
        self.tensorMelgrams = []
        
        
        self.info = " VoiceSamples Object successfully created "
        self.Log.info(self.info)
        
        
        if Automatic:
            self.LoadSoundSamples()
            self.ChopToOneSecFragments()
            self.ChopedSignalsToTenosor()
        
    def __len__(self):
        return len(self.tensorMelgrams)
    
    def __getitem__(self, idx):
        if self.tensorMelgrams:
            return self.tensorMelgrams[idx]

    def LoadSoundSamples(self):
    
        n = 1

        while(True):
            try:
                if  self.samples_path:
                    path =  self.samples_path + self.core_name + str(n)
                else:
                    path = self.core_name + str(n)

                soundSample, sampleRate = lr.load(path)

                n += 1
                self.soundSamples.append(soundSample)
                self.sampleRate.append(sampleRate) 
                self.path.append(path)

                self.info = " Sample : " + path + " : successfully added"
                self.Log.info(self.info)

            except FileNotFoundError:
                if self.soundSamples:
                    self.info = "That's the end of database : " + str(n-1) + " : Samples added"
                    self.Log.info(self.info)
                    n = 0
                    
                    return self.soundSamples, self.sampleRate, self.path

                else:
                    self.Log.exception("Files are missing")
                    n = 0

                break

            except Exception as ex:      
                self.Log.exception("Unexpected error")
                break
        
    def getSoundSample(self, idx):
        return self.soundSamples[idx], self.sampleRate[idx]
    
    def getSoundSampleLen(self):
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                return len(self.soundSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
    
    def ChopToOneSecFragments(self):
        
        # TODO: make shure user goes step by step 
        
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                for idx in range(len(self.soundSamples)):
                    
                    soundSample = self.soundSamples[idx]
                    sr = self.sampleRate[idx]
                    
                    frag_max = math.trunc(len(soundSample)/float(sr))
                    step = math.trunc(sr/2);
                    last_sample = len(soundSample)

                    for frag in range(frag_max*2):
                        start = step * frag
                        stop = start + sr
                        if sr<len(soundSample):
                            if self.checkIfNotNoise(soundSample[start:stop]):
                                self.chopedSamples.append(soundSample[start:stop])
                                self.chopedSr.append(sr)
                                self.info = self.path[idx] + " : " + str(frag+1) + " : successfully choped"
                                self.Log.info(self.info)
                            else:
                                self.info = self.path[idx] + " : " + str(frag+1) + " : NOISE!"
                                self.Log.info(self.info)
                        else:
                            self.Log.warning("Something went wrong")
                            
                    if self.checkIfNotNoise(soundSample[last_sample-sr:last_sample]):
                         # incuding samples cuted by math.trunc() 
                        self.chopedSamples.append(soundSample[last_sample-sr:last_sample])
                        self.chopedSr.append(sr)
                        self.info = self.path[idx] +  " : "  + str(frag_max*2+1) + " : successfully choped"
                        self.Log.info(self.info)
                    else:
                        self.info = self.path[idx] + " : "  + str(frag+1) + " : NOISE!"
                        self.Log.info(self.info)
                
                if self.chopedSamples:
                    self.Log.info("Sucessfully choped all loaded signals and eliminated the noise!")
                    return self.chopedSamples, self.chopedSr 
                    
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
            
    def getChoped(self, idx):
        return self.chopedSamples[idx], self.chopedSr[idx]
        
    def getChopedLen(self):
        try:
            if len(self.chopedSamples) == len(self.chopedSr):
                    return len(self.chopedSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
            
        
    def ChopedSignalsToTenosor(self):
        
        # TODO: make shure user goes step by step 
        
        try:
        
            if len(self.chopedSamples) == len(self.chopedSr):
                for idx in range(len(self.chopedSamples)):

                    # hop length adjusted
                    STFT_signal = np.abs(lr.stft(self.chopedSamples[idx], n_fft = 512, hop_length = round(self.chopedSr[idx]/256))) 
                    STFT_signal = lr.power_to_db(STFT_signal**2,ref=np.max)

                    Melgram = STFT_signal[0:256,0:256]
                    TMelgram = torch.tensor(Melgram)
                    self.tensorMelgrams.append(TMelgram)
                    
                    self.info = " " + self.samples_path +  " : ChopedSample " + str(idx) + " : " + " : converted to tensor"
                    self.Log.info(self.info)
                
                if self.tensorMelgrams:
                    self.Log.info("Sucessfully converted all ChopedSamples to Tensors!")
                    return self.tensorMelgrams
                
            else:
                self.Log.warning("Lists: chopedSamples and chopedSr are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
                
    
    
    def checkIfNotNoise(self, chopedSample):
    
        chopedSamplePow2 = []

        for n in range(len(chopedSample)):
            chopedSamplePow2.append(chopedSample[n]**2)
        sk = sum(chopedSamplePow2)
        if sk > self.noiseThreshold:
            return True 
        else:
            return False

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VoiceRecogModel(nn.Module):

    def __init__(self):
        super(VoiceRecogModel, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*62*62, 120)  # ?? from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
import torch.optim as optim

# create dataset : loading and processing samples to tensors
vsInput = VoiceSamplesInput()


INFO:root: VoiceSamples Object successfully created 
INFO:root: Sample : database/Krystian/vsKrystian1 : successfully added
INFO:root: Sample : database/Krystian/vsKrystian2 : successfully added
INFO:root:That's the end of database : 2 : Samples added
INFO:root:database/Krystian/vsKrystian1 : 1 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 2 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 3 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 4 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 5 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 6 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 7 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 8 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 9 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 10 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 11 : successfully choped
INFO:root:database/K

INFO:root:database/Krystian/vsKrystian2 : 103 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 104 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 105 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 106 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 107 : NOISE!
INFO:root:database/Krystian/vsKrystian2 : 108 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 109 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 110 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 111 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 112 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 113 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 114 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 115 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 116 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 117 : successfully 

INFO:root: database/Krystian/ : ChopedSample 81 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 82 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 83 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 84 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 85 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 86 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 87 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 88 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 89 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 90 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 91 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 92 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 93 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 94 :  

INFO:root:database/Nicia/vsNicia1 : 56 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 57 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 58 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 59 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 60 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 61 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 62 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 63 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 64 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 65 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 66 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 67 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 68 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 69 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 70 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 71 : successfully choped
INFO:root:database/Nicia

INFO:root:database/Nicia/vsNicia2 : 113 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 114 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 115 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 116 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 117 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 118 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 119 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 120 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 121 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 122 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 123 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 124 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 125 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 126 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 127 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 128 : successfully choped
INFO:roo

INFO:root: database/Nicia/ : ChopedSample 44 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 45 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 46 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 47 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 48 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 49 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 50 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 51 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 52 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 53 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 54 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 55 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 56 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 57 :  : converted to tensor
INFO:root: database/

INFO:root: database/Nicia/ : ChopedSample 161 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 162 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 163 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 164 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 165 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 166 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 167 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 168 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 169 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 170 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 171 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 172 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 173 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 174 :  : converted to tensor
INFO:r

In [9]:
# create net from VoiceRecogModel
net = VoiceRecogModel()

# loss function (using function implemented in pytorch)
criterion = nn.MSELoss()

# create your optimizer (basic optimizer)
# setting learning rate 
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [10]:
import logging
logging.basicConfig(level=logging.INFO)
trainLog = logging.getLogger()

# Training Loop

for k in range(len(vsInput)):  
    
    vs, target = vsInput[k]
    
    optimizer.zero_grad()   # zero the gradient buffers
    input = vs.view(-1,1,256,256)
    output = net(input)
    print(target)
    print(output)
    loss = criterion(output, target)
    print(loss)
    loss.backward()
    optimizer.step()    # Does the update
    info = "Training " + str(k)+"/"+str(len(vsInput))+" done"
    trainLog.info(info)

tensor([[1., 0.]])
tensor([[-0.1174, -0.7282]], grad_fn=<AddmmBackward>)
tensor(0.8894, grad_fn=<MseLossBackward>)


INFO:root:Training 0/286 done


tensor([[0., 1.]])
tensor([[13.4370,  3.5886]], grad_fn=<AddmmBackward>)
tensor(93.6269, grad_fn=<MseLossBackward>)


INFO:root:Training 1/286 done


tensor([[1., 0.]])
tensor([[-3.0865, -0.0514]], grad_fn=<AddmmBackward>)
tensor(8.3511, grad_fn=<MseLossBackward>)


...

INFO:root:Training 283/286 done


tensor([[1., 0.]])
tensor([[0.5663, 0.3159]], grad_fn=<AddmmBackward>)
tensor(0.1440, grad_fn=<MseLossBackward>)


INFO:root:Training 284/286 done


tensor([[0., 1.]])
tensor([[0.1688, 0.6858]], grad_fn=<AddmmBackward>)
tensor(0.0636, grad_fn=<MseLossBackward>)


INFO:root:Training 285/286 done
